In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import os
import joblib


In [3]:
data = pd.read_csv('../data/loan_approval_dataset.csv')

In [13]:
data.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,Total assets
0,1,2,1,0,0.969072,0.755102,12,778,0.085616,0.907216,0.575835,0.544218,1,0.557032
1,2,0,0,1,0.402062,0.303571,8,417,0.095890,0.113402,0.218509,0.224490,0,0.183832
2,3,3,1,0,0.917526,0.750000,20,506,0.246575,0.231959,0.848329,0.870748,0,0.634551
3,4,3,1,0,0.824742,0.775510,8,467,0.626712,0.170103,0.591260,0.537415,0,0.579181
4,5,5,0,1,0.989691,0.609694,20,382,0.428082,0.422680,0.748072,0.340136,0,0.604651


In [5]:
# Strip leading/trailing spaces from column names
data.columns = data.columns.str.strip()

# Access the 'education' column
data["education"].value_counts()

education
Graduate        2144
Not Graduate    2125
Name: count, dtype: int64

In [6]:
data['loan_status'] = data['loan_status'].str.strip()
data['loan_status'] = data['loan_status'].map({'Approved': 1, 'Rejected': 0})

In [7]:
data["education"]=data["education"].str.strip()
data['education'] = data['education'].map({'Graduate': 1, 'Not Graduate': 0})

In [8]:
data["self_employed"]=data["self_employed"].str.strip()
data["self_employed"]=data["self_employed"].map({'Yes': 1, 'No': 0})

In [9]:
data["education"].head()

0    1
1    0
2    1
3    1
4    0
Name: education, dtype: int64

In [10]:
data["Total assets"]=data["residential_assets_value"]+data["commercial_assets_value"]+data["luxury_assets_value"]+data["bank_asset_value"]

In [11]:
correlation_total = data['loan_status'].corr(data['Total assets'])
print(f"Correlation between loan status and CIBIL score: {correlation_total}")

Correlation between loan status and CIBIL score: -0.01128142674264055


In [12]:
scaler = MinMaxScaler()
data[['income_annum', 'loan_amount', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'Total assets']] = scaler.fit_transform(data[['income_annum', 'loan_amount', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'Total assets']])